In [1]:
from utils import read_csv, list_files
import pandas as pandas

In [2]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

In [3]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '../secrets/client_secrets.json'

In [4]:
from storage import GStorage
bucket = GStorage('car_comparator')

In [5]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

../secrets/client_secrets.json


In [6]:
df_cars_check = read_csv('gstorage','data/flexicar/make_and_model_links.csv',bucket)
df_cars_check

,car_make,id1,car_model,id2,link,Name,year_min,price_max,year_max,price_min,km_max
0,Audi,a3,A3,audi,https://www.flexicar.es/audi/a3/segunda-mano/,Audi A3,2014,30000,2022,10000,125000
1,Audi,a4,A4,audi,https://www.flexicar.es/audi/a4/segunda-mano/,Audi A4,2014,30000,2022,10000,125000
2,BMW,x1,X1,bmw,https://www.flexicar.es/bmw/x1/segunda-mano/,BMW X1,2014,30000,2022,10000,125000
3,Renault,talisman,Talisman,renault,https://www.flexicar.es/renault/talisman/segun...,Renault Talisman,2014,30000,2022,10000,125000
4,Volkswagen,golf,Golf,volkswagen,https://www.flexicar.es/volkswagen/golf/segund...,Volkswagen Golf,2014,30000,2022,10000,125000
5,Volkswagen,t-roc,T-Roc,volkswagen,https://www.flexicar.es/volkswagen/t-roc/segun...,Volkswagen T-Roc,2014,30000,2022,10000,125000
6,Volkswagen,tiguan,Tiguan,volkswagen,https://www.flexicar.es/volkswagen/tiguan/segu...,Volkswagen Tiguan,2014,30000,2022,10000,125000
7,Mercedes-Benz,clase-a,A-Class,mercedes-benz,https://www.flexicar.es/mercedes-benz/clase-a/...,Mercedes-Benz A-Class,2014,30000,2022,10000,125000
8,Mercedes-Benz,clase-gla,GLA-Class,mercedes-benz,https://www.flexicar.es/mercedes-benz/clase-gl...,Mercedes-Benz GLA-Class,2014,30000,2022,10000,125000


In [7]:
path= list_files('gstorage','data/flexicar/final_data',bucket)[-2]
path

'data/flexicar/final_data/scrap_flexicar_20220508_164703.csv'

In [8]:
df = read_csv('gstorage',list_files('gstorage','data/flexicar/final_data',bucket)[-2],bucket)
df[['car_make','car_model']].value_counts()

car_make       car_model
Mercedes-Benz  A-Class      48
Volkswagen     Golf         47
Mercedes-Benz  GLA-Class    27
BMW            X1           24
Volkswagen     Tiguan       24
Audi           A4           19
Volkswagen     T-Roc        18
Renault        Talisman     14
Audi           A3            1
dtype: int64

In [35]:
def clean_data(data):
    data['price'] = data['Price'].str.replace(',', '')
    data['price'] = data['price'].str.extract('(\d+)').astype(int, errors='ignore')

    data = data[~data['price'].isna()]
    data['price'] = data['price'].astype(int)
    
    # data['year'] = data['First Registration'].apply(lambda x : str(x).split('/')[1])
    # data['month'] = data['First Registration'].apply(lambda x : str(x).split('/')[0])
    data['price_category'] = data['Price'].str.extract('([a-zA-Z]+)')
    return data

In [ ]:
storage_type = 'gstorage'

fileList = list_files(storage_type, "data/mobile_de/make_model_ads_data/", bucket)
len(fileList)

In [11]:
name = 'mobile_de'
storage_type = 'gstorage'
import tqdm
import pandas as pd

def concatenate_dfs(indir, save_to_csv,bucket,storage_type):
    
    fileList = list_files(storage_type, indir, bucket)

    dfs = [read_csv(storage_type, path, bucket) for path in fileList[:2000]]

    output_file = pd.concat(dfs)

    cols = list(set(output_file.columns) - set(['download_date_time']))
    output_file = output_file.drop_duplicates(subset=cols,keep='last')

    if save_to_csv:
        try:
            write_csv(storage_type,output_file,f"data/mobile_de/make_model_ads_concatinated.csv",bucket)
        except:
            print('')

    output_file.reset_index(drop=True, inplace=True)

    return(output_file)

def merge_make_model_keep_latest(data,make_model_ads_data_latest):
    latest_scrape = data.groupby(['link'], dropna=True).agg(number_of_ads=('link', 'count'), latest_scrape=('download_date_time', 'max'))
    latest_scrape = latest_scrape.reset_index()

    data = pd.merge(data, latest_scrape[['link', 'latest_scrape']], how = 'left', left_on = ['link'], right_on = ['link'])
    data = data.reset_index(drop=True)
    # keep rows where download_date_time is equal to latest_scrape
    data = data[data['download_date_time'] == data['latest_scrape']]
    data = data.reset_index(drop=True)
    # drop the latest_scrape column
    data = data.drop(columns = ['latest_scrape'])
    make_model_ads_data_latest = make_model_ads_data_latest[['make_model_link','car_make', 'car_model']].drop_duplicates(keep='first')

    data = pd.merge(data, make_model_ads_data_latest, how = 'inner', left_on = 'make_model_link', right_on = 'make_model_link')

    return data

individual_ads_data = clean_data(concatenate_dfs(f"data/mobile_de/make_model_ads_data/",  True,bucket,storage_type)).drop_na

In [49]:
make_model_ads_data = read_csv(storage_type,"data/mobile_de/make_model_ads_links_concatinated.csv", bucket)

individual_ads_data = clean_data(individual_ads_data.dropna())

individual_ads_data = concatenate_dfs("data/mobile_de/make_model_ads_data/",  True,bucket,storage_type)

ads_df = merge_make_model_keep_latest(data = individual_ads_data, make_model_ads_data_latest = make_model_ads_data_latest)
ads_df_clean = clean_data(data = ads_df)

now = datetime.now() 
datetime_string = str(now.strftime("%Y%m%d_%H%M%S"))
ads_df_clean['audit_date'] = datetime_string
df_mb = ads_df_clean.copy()

NameError: name 'merge_make_model_keep_latest' is not defined

In [29]:
df_mb['model'].dropna().value_counts()

Audi A4    345
Audi A3    312
Name: model, dtype: int64

In [43]:
df_mb['price'].min()

12590

In [45]:
df_mb.columns

Index(['Price', 'Category', 'Mileage', 'Cubic Capacity', 'Power', 'Fuel',
       'Fuel Consumption ', 'Energy efficiency class', 'Number of Seats',
       'Gearbox', 'Emissions Sticker', 'First Registration', 'Parking sensors',
       'Colour', 'Interior Design', 'link', 'make_model_link', 'model',
       'download_date_time', 'price', 'price_category'],
      dtype='object')

In [44]:
df.columns

Index(['First Registration', 'Mileage', 'Fuel', 'Number of Seats',
       'Cubic Capacity', 'Colour', 'Gearbox', 'Consume', 'IVA', 'Price',
       'model_type', 'location', 'Emissions Sticker', 'Interior', 'price',
       'price_category', 'link', 'download_date_time', 'ad_link', 'car_make',
       'car_model', 'audit_date'],
      dtype='object')

In [ ]:
set(list(df.columns)) - (set(list(df.columns)) - set(list(df_mb.columns)))